# Elo Merchant Category Recommendation

### ※ 프로젝트 개요
####   
####   
### ● 고객 충성도 예측  
### ● Elo : 브라질 최대 결제 브랜드 중 하나 
###   
### 목표 : 탐색적 데이터 분석
###   

## 1. 데이터 살펴보기

In [1]:
import pandas as pd

tr = pd.read_excel('./Data_Dictionary.xlsx', sheet_name = 'train', header = 1)
hs = pd.read_excel('./Data_Dictionary.xlsx', sheet_name = 'history', header = 1)
mp = pd.read_excel('./Data_Dictionary.xlsx', sheet_name = 'new_merchant_period', header = 1)
mc = pd.read_excel('./Data_Dictionary.xlsx', sheet_name = 'merchant', header = 1)

train = pd.read_csv('./train.csv')
hist = pd.read_csv('./historical_transactions.csv')
nmp = pd.read_csv('./new_merchant_transactions.csv')
mer = pd.read_csv('./merchants.csv')
test = pd.read_csv('./test.csv')

#### train.csv
 * key 값으로 사용되는 card_id와 target을 제외하면 변수는 4개, 그 중에서 하나는 날짜이다.
 * card_id를 키값으로 'history'와 'merchant period' 데이터와 연결되어 있다. 

    0.  card_id : 고유 카드 식별자
    1.  first_active_month : 'YYY-MM', 첫 구매월
    2.  feature_1 : 익명화된 카드 범주형 특징
    3.  feature_2 : 익명화된 카드 범주형 특징
    4.  feature_3 : 익명화된 카드 범주형 특징
    5.  target : 과거 및 평가 기간 2개월 후 계산된 로열티 수치

In [2]:
tr

,Columns,Description
0,card_id,Unique card identifier
1,first_active_month,"'YYYY-MM', month of first purchase"
2,feature_1,Anonymized card categorical feature
3,feature_2,Anonymized card categorical feature
4,feature_3,Anonymized card categorical feature
5,target,Loyalty numerical score calculated 2 months af...


In [3]:
print(train.shape)
train.head()

(201917, 6)


,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


#### historical_transactions.csv
 * card_id에 따른 상품 구매내역이 있다. 여기서 key값 'merchant_id'는 'merchant period', 'merchant'와 이어진다.
 * 대부분의 데이터는 익명처리화 
 * 상품, 구매일, 승인여부, 할부, 금액, 가맹점 등의 정보가 있다.
 * 2천 9백만 건의 방대한 데이터를 가지고 있다.
 * 동일 card id에 따른 각각의 거래기록이 있다.
 
    0.  card_id : 카드 식별자
    1.  month_lag : 기준일보다 월 지연
    2.  purchase_date : 구매일자
    3.  authorized_flag : 승인된 경우 'Y', 거부된 경우 'N'
    4.  category_3 :익명으로 분류된 범주
    5.  installments : 구매 할부 수
    6.  category_1 : 익명으로 분류된 범주
    7.  merchant_category_id : 가맹점 카테고리 식별자(익명됨)
    8.  subsector_id: 가맹점 카테고리 그룹 식별자(익명됨)
    9.  merchant_id : 가맹점 식별자(익명)
    10. purchase_amount : 정상매입금액
    11. city_id : 도시 식별자(익명)
    12. state_id :상태 식별자(익명됨)
    13. category_2 : 익명으로 분류된 범주

In [4]:
hs

,Columns,Description
0,card_id,Card identifier
1,month_lag,month lag to reference date
2,purchase_date,Purchase date
3,authorized_flag,"Y' if approved, 'N' if denied"
4,category_3,anonymized category
5,installments,number of installments of purchase
6,category_1,anonymized category
7,merchant_category_id,Merchant category identifier (anonymized )
8,subsector_id,Merchant category group identifier (anonymized )
9,merchant_id,Merchant identifier (anonymized)


In [5]:
print(hist.shape)
hist.head()

(29112361, 14)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


#### new_merchant_transactions.csv
 * historical_transactions.csv 데이터 정보와 같다.
 * 최신 상품 거래 기록(2018년 위주)을 가지고 있으며 마찬가지로 동일 card_id에 여러 거래기록이 남아있다. 

    0.  card_id : 카드 식별자
    1.  month_lag : 기준일보다 월 지연
    2.  purchase_date : 구매일자
    3.  authorized_flag : 승인된 경우 'Y', 거부된 경우 'N'
    4.  category_3 :익명으로 분류된 범주
    5.  installments : 구매 할부 수
    6.  category_1 : 익명으로 분류된 범주
    7.  merchant_category_id : 가맹점 카테고리 식별자(익명됨)
    8.  subsector_id: 가맹점 카테고리 그룹 식별자(익명됨)
    9.  merchant_id : 가맹점 식별자(익명)
    10. purchase_amount : 정상매입금액
    11. city_id : 도시 식별자(익명)
    12. state_id :상태 식별자(익명됨)
    13. category_2 : 익명으로 분류된 범주

In [6]:
mp

,Columns,Description
0,card_id,Card identifier
1,month_lag,month lag to reference date
2,purchase_date,Purchase date
3,authorized_flag,"Y' if approved, 'N' if denied"
4,category_3,anonymized category
5,installments,number of installments of purchase
6,category_1,anonymized category
7,merchant_category_id,Merchant category identifier (anonymized )
8,subsector_id,Merchant category group identifier (anonymized )
9,merchant_id,Merchant identifier (anonymized)


In [7]:
print(nmp.shape)
nmp.head()

(1963031, 14)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


#### merchants.csv
 * 상품정보 데이터로 상품의 id, 그룹 id, 카테고리 id 등의 식별 데이터가 있다.
 * 상품 id는 위 card_id 별 거래기록이 있던 것과는 다르게 고유의 id이다.
 * 3/6/12 개월의마다의 평균 판매, 구매, 활성 월에 대한 데이터가 있다.
 * 특이점으로는 category_3 은 없고, 1,2,4만 존재한다.

    0.  merchant_id :             고유 가맹점 식별자
    1.  merchant_그룹_id :        상인 그룹(익명)
    2.  merchant_category_id :    가맹점 범주의 고유 식별자(익명됨)
    3.  subsector_id :            상인 카테고리 그룹(익명)
    4.  숫자_1 :                  익명으로 하는 조치
    5.  숫자_2 :                  익명으로 하는 조치
    6.  category_1 : 익명으로 분류된 범주
    7.  most_recent_sales_range : 지난 달 수익(화폐 단위) 범위 - A > B > C > D > E
    8.  most_recent_purchase_range : 지난 달 거래량 범위 - A > B > C > D > E
    9.  avg_sales_lag3 : 지난 3개월 동안의 월평균 매출액을 지난 활성 월의 수익으로 나눈 값
    10. avg_purchase_lag3 : 지난 3개월 동안의 월별 트랜잭션 평균을 지난 활성 월의 트랜잭션으로 나눈 값
    11. active_months_lag3 : 지난 3개월 이내 활성월수
    12. avg_sales_lag6 : 지난 6개월 동안의 월평균 매출액을 지난 활성 월의 수익으로 나눈 값
    13. avg_purchase_lag6 : 지난 6개월 동안의 월평균을 지난 활성월의 트랜잭션으로 나눈 값
    14. active_months_lag6 : 지난 6개월 이내 활성월수
    15. avg_sales_lag12 : 지난 12개월 동안의 월평균 매출액을 지난 활성 월의 수익으로 나눈 값
    16. avg_purchase_lag12 : 지난 12개월 동안의 월평균을 지난 활성월의 거래로 나눈 값
    17. active_months_lag12 : 지난 12개월 내 활성월 수
    18. category_4 : 익명으로 분류된 범주
    19. city_id : 도시 식별자(익명)
    20. state_id : 상태 혹은 주 식별자(익명됨)
    21. category_2 익명으로 분류된 범주

In [8]:
mc

,Columns,Description
0,merchant_id,Unique merchant identifier
1,merchant_group_id,Merchant group (anonymized )
2,merchant_category_id,Unique identifier for merchant category (anony...
3,subsector_id,Merchant category group (anonymized )
4,numerical_1,anonymized measure
5,numerical_2,anonymized measure
6,category_1,anonymized category
7,most_recent_sales_range,Range of revenue (monetary units) in last acti...
8,most_recent_purchases_range,Range of quantity of transactions in last acti...
9,avg_sales_lag3,Monthly average of revenue in last 3 months di...


In [9]:
print(mer.shape)
mer.head()

(334696, 22)


,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


####   
####   

## 2.  카드데이터 살펴보기

#### card_id 'C_ID_92a2005557'를 hist, nmp에서 어떻게 있는지 살펴본다.

In [10]:
train[train['card_id'] == 'C_ID_92a2005557']

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283


In [25]:
hist[hist['card_id'] == 'C_ID_92a2005557'].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
12454872,Y,C_ID_92a2005557,69,N,0,A,195,M_ID_cc72dac895,-3,-0.506033,2017-11-25 16:37:11,1.0,9,34
12454873,Y,C_ID_92a2005557,69,N,0,A,278,M_ID_c9c0f1cb3f,-7,-0.722114,2017-07-16 12:21:00,1.0,9,37
12454874,Y,C_ID_92a2005557,69,N,0,A,683,M_ID_36f9e41993,-6,-0.726171,2017-08-10 21:01:16,1.0,9,34
12454875,Y,C_ID_92a2005557,69,N,0,A,309,M_ID_b790230ee9,-2,-0.610167,2017-12-15 12:06:58,1.0,9,21
12454876,Y,C_ID_92a2005557,69,N,0,A,278,M_ID_ab67f6c982,-7,-0.307232,2017-07-08 15:13:18,1.0,9,37


In [16]:
print('the oldest day : ', hist[hist['card_id'] == 'C_ID_92a2005557']['purchase_date'].min())

the oldest day :  2017-06-27 14:18:08


In [26]:
nmp[nmp['card_id'] == 'C_ID_92a2005557'].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1246676,Y,C_ID_92a2005557,19,N,0,A,80,M_ID_98b342c0e3,1,-0.491457,2018-03-30 10:30:32,1.0,9,37
1246677,Y,C_ID_92a2005557,69,N,0,A,80,M_ID_28a21ea733,2,-0.666561,2018-04-06 10:55:08,1.0,9,37
1246678,Y,C_ID_92a2005557,19,N,0,A,278,M_ID_20dce166b9,1,-0.635246,2018-03-29 15:38:10,1.0,9,37
1246679,Y,C_ID_92a2005557,69,N,0,A,842,M_ID_f7796994d1,2,-0.581180,2018-04-09 13:56:50,1.0,9,37
1246680,Y,C_ID_92a2005557,276,N,0,A,783,M_ID_0703c10cc0,1,-0.296112,2018-03-12 16:49:03,1.0,9,19


In [20]:
print('the newest day : ', nmp[nmp['card_id'] == 'C_ID_92a2005557']['purchase_date'].max())

the newest day :  2018-04-29 11:23:05


In [24]:
oldest = pd.to_datetime(hist[hist['card_id'] == 'C_ID_92a2005557']['purchase_date'].min(), format = '%Y-%m-%d')
newest = pd.to_datetime(nmp[nmp['card_id'] == 'C_ID_92a2005557']['purchase_date'].max(), format = '%Y-%m-%d') 
newest - oldest

Timedelta('305 days 21:04:57')

#### 305일 동안 총 283개의 거래기록 